# 811s A&M
### outputs
* Gets both smartsheets and combines them - Named: To add more APNs.xlsx
* Gets the vlookup for the 811 A&M tracker - Named: SmartSheets Data-811 vlookups

In [9]:
#libraies:
import pandas as pd
import numpy as np
import smartsheet
import glob
import os

#Variables
# random
excel = '.xlsx'
csv = '.csv'

# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API

Central.errors_as_exceptions(True)

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)

#download smartsheets to any folder you need
# Central.Sheets.get_sheet_as_excel(Central_id, download_path=r"C:\Users\jacque.trahan\Downloads")
# South.Sheets.get_sheet_as_excel(South_id, download_path=r"C:\Users\jacque.trahan\Downloads") 


#Google Sheets
#Sheet ID comes right after /d/ in the URL
Google_sheet_ID= '1O2dmdhEo5hJVAgRVDl4M4n3_v-KqHS56q3OZVdpZANU'
# name of tab you want to use
Google_sheets_name = 'Tree RTR Tracker'

# this is tricky. after /export? we must put format=csv#id={Sheet Name}   -> manually to make this work
url = f'https://docs.google.com/spreadsheets/d/{Google_sheet_ID}/export?format=csv#id={Google_sheets_name}'

# Function to read in column titles and row vales and make out data frames
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

combined = pd.concat([Central_SS,South_SS])

# imporved Smart sheets load in above.
### code below no longer in use

In [10]:
# path = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
# file_type = ".xlsx"
# load_it = glob.glob(path + file_type)
# df = pd.read_excel(max(load_it, key=os.path.getctime))

In [11]:
# path2 = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
# loadit2 = glob.glob(path2 + file_type)
# df2 = pd.read_excel(max(loadit2, key=os.path.getctime))

In [12]:
# combined = pd.concat([df, df2])

In [13]:
# erosion = combined.copy()
# erosion = erosion[['APN_ROW Segment','Street #', 'Street Name', 'County',
#                   'Debris Crew', 'Nesting Bird Survey Date']]

# erosion[(erosion['Debris Crew'] == 'Erosion Control') & (erosion['Nesting Bird Survey Date'].isnull())].to_excel('Erosion no bird survey.xlsx')

In [14]:
# combined['Debris Crew'].value_counts()

In [15]:
combined = combined[(~combined['APN_ROW Segment'].isnull())]

combined.to_excel("To add more APNs.xlsx")

# RTR ticket date

In [17]:
path = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
load_rtr = glob.glob(path+excel)
df_RTR = pd.read_excel(max(load_rtr, key=os.path.getctime))

df_RTR['Zone Name'] = df_RTR['Zone Name'].str[:3] + "-" + df_RTR['Zone Name'].str[3:6] + "-" + df_RTR['Zone Name'].str[6:]
df_RTR['Service Code'] = df_RTR['Service Code'].str.strip()

In [18]:
#RTR data set up
sign = df_RTR.copy()
sign = sign[['Zone Name','Is Void', 'GIS Zone1', 'Service Code', 'Unit Count','Ticket Number', 'GPSNorth', 'GPSWest', 'End Time']]
sign = sign[(sign['Service Code'].isin(["67A"])) & ~(sign['GIS Zone1'].isin(['NORTHERN DIVISION'])) & (sign['Is Void']== False)]


# For 811 A&M Central Tracker vlookups

In [19]:
vlook_811 = combined[['APN_ROW Segment','Street #', 'Street Name', 'County','811 Called (A&M)', 'Structural Status']].copy()
vlook_811['811 Called (A&M)'] = np.where(vlook_811['811 Called (A&M)'] == 1, "Completed", "Incomplete")
vlook_811 = vlook_811[~(vlook_811['APN_ROW Segment'].isnull())] # takes out blank APN rows


# Mergre RTR and SS

In [20]:
final = vlook_811.merge(sign,
                       left_on='APN_ROW Segment',
                       right_on='Zone Name',
                       how='outer')

final = final[['APN_ROW Segment','Street #', 'Street Name', 'County','811 Called (A&M)', 'Structural Status',
              'Ticket Number', 'GPSNorth', 'GPSWest', 'End Time','GIS Zone1',]]

In [21]:
# this will up date the 811 A&M vlookup tracker
final.set_index('APN_ROW Segment', inplace=True)
final.to_excel('SmartSheets Data-811 vlookups.xlsx')